In [2]:
print("hello world")

hello world


In [3]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Load Excel files
descriptions_df = pd.read_excel('product_descriptions.xlsx')
brands_df = pd.read_excel('brands.xlsx')

# Rename columns if needed
descriptions_df.columns = ['Description', 'data_key']
brands_df.columns = ['Brand']

def clean_text(text):
    text = str(text)  # Convert to string to avoid errors
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)  # remove parentheses and content inside
    text = re.sub(r"[’'‘`]", '', text)     # remove apostrophes and variants
    # remove common company suffixes
    text = re.sub(r'\b(inc|incorporated|ltd|llc|corp|co|company)\b', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)   # remove punctuation
    text = re.sub(r'\s+', ' ', text)       # replace multiple spaces with single space
    return text.strip()

# Filter out empty or NaN brand entries before cleaning
brands_df = brands_df.dropna(subset=['Brand'])

cleaned_brands = [clean_text(b) for b in brands_df['Brand']]
cleaned_to_original = dict(zip(cleaned_brands, brands_df['Brand']))

SIMILARITY_THRESHOLD = 75
TOP_N = 5

matched_brands_per_description = []

print("Starting matching process for top 5 brand matches...")

for desc in descriptions_df['Description'].astype(str):
    desc_lower = desc.lower()
    
    # Attempt to extract substring after " by "
    if ' by ' in desc_lower:
        after_by = desc_lower.split(' by ', 1)[1]
        after_by_clean = clean_text(after_by)
        
        results = process.extract(
            query=after_by_clean,
            choices=cleaned_brands,
            scorer=fuzz.token_set_ratio,
            limit=TOP_N
        )
        
        filtered_matches = [cleaned_to_original[match[0]] for match in results if match[1] >= SIMILARITY_THRESHOLD]
        
        if not filtered_matches:
            desc_clean = clean_text(desc)
            results = process.extract(
                query=desc_clean,
                choices=cleaned_brands,
                scorer=fuzz.token_set_ratio,
                limit=TOP_N
            )
            filtered_matches = [cleaned_to_original[match[0]] for match in results if match[1] >= SIMILARITY_THRESHOLD]
    else:
        desc_clean = clean_text(desc)
        results = process.extract(
            query=desc_clean,
            choices=cleaned_brands,
            scorer=fuzz.token_set_ratio,
            limit=TOP_N
        )
        filtered_matches = [cleaned_to_original[match[0]] for match in results if match[1] >= SIMILARITY_THRESHOLD]
    
    matched_brands_per_description.append(', '.join(filtered_matches))

# Add matched brands as a new column
descriptions_df['Matched_Brands'] = matched_brands_per_description

# Reorder columns to have data_key first
output_df = descriptions_df[['data_key', 'Description', 'Matched_Brands']]

# Save to Excel
output_df.to_excel('brand_match_results_top5_with_key.xlsx', index=False)

print("Matching complete! Results saved to 'brand_match_results_top5_with_key.xlsx'.")


Starting matching process for top 5 brand matches...
Matching complete! Results saved to 'brand_match_results_top5_with_key.xlsx'.


In [4]:
import pandas as pd
import re

# Load the Excel file which contains: data_key, Description, Matched_Brands
df = pd.read_excel('brand_match_results_top5_with_key.xlsx')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)             # remove parentheses and content inside
    text = re.sub(r"[’'‘`]", '', text)                # remove apostrophes and variants
    text = re.sub(r'\b(inc|incorporated|ltd|llc|corp|co|company)\b', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)              # remove punctuation
    text = re.sub(r'\s+', ' ', text)                   # normalize spaces
    return text.strip()

def clean_brand_name(brand):
    brand = re.sub(r'\([^)]*\)', '', brand)  # remove parentheses and content inside
    return clean_text(brand).strip()

def brand_in_description(brand, description):
    description_tokens = set(clean_text(description).split())
    brand_tokens = set(brand.split())
    return brand_tokens.issubset(description_tokens)

def split_brands(s):
    # Split on commas NOT within parentheses
    pattern = r',\s*(?![^()]*\))'
    return re.split(pattern, s)

def filter_matched_brands(description, matched_brands_str):
    if pd.isna(matched_brands_str) or matched_brands_str.strip() == '':
        return ''
    
    matched_brands_raw = split_brands(matched_brands_str)
    filtered_brands = []
    for raw_brand in matched_brands_raw:
        brand = clean_brand_name(raw_brand)
        if brand_in_description(brand, description):
            filtered_brands.append(raw_brand.strip())
    return ', '.join(filtered_brands)

# Apply filtering row-wise
df['Filtered_Brands'] = df.apply(
    lambda row: filter_matched_brands(row['Description'], row['Matched_Brands']),
    axis=1
)

# Save output with data_key included
output_df = df[['data_key', 'Description', 'Matched_Brands', 'Filtered_Brands']]
output_df.to_excel('brand_match_results_filtered_with_key.xlsx', index=False)

print("Filtering complete! Results saved to 'brand_match_results_filtered_with_key.xlsx'.")


Filtering complete! Results saved to 'brand_match_results_filtered_with_key.xlsx'.


In [2]:
import pandas as pd
import re

# Load the Excel file which contains: data_key, Description, Matched_Brands
df = pd.read_excel('brand_match_results_filtered_with_key.xlsx')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)             # remove parentheses and content inside
    text = re.sub(r"[’'‘`]", '', text)                # remove apostrophes and variants
    text = re.sub(r'\b(inc|incorporated|ltd|llc|corp|co|company)\b', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)              # remove punctuation
    text = re.sub(r'\s+', ' ', text)                   # normalize spaces
    return text.strip()

def clean_brand_name(brand):
    brand = re.sub(r'\([^)]*\)', '', brand)  # remove parentheses and content inside
    return clean_text(brand).strip()

def brand_in_description(brand, description):
    description_clean = clean_text(description)
    desc_tokens = description_clean.split()
    half_len = max(1, len(desc_tokens) // 2)
    truncated_tokens = set(desc_tokens[:half_len])
    brand_tokens = set(brand.split())
    return brand_tokens.issubset(truncated_tokens)

def split_brands(s):
    # Split on commas NOT within parentheses
    pattern = r',\s*(?![^()]*\))'
    return re.split(pattern, s)

def filter_matched_brands(description, matched_brands_str):
    if pd.isna(matched_brands_str) or matched_brands_str.strip() == '':
        return ''
    
    matched_brands_raw = split_brands(matched_brands_str)
    filtered_brands = []
    for raw_brand in matched_brands_raw:
        brand = clean_brand_name(raw_brand)
        if brand_in_description(brand, description):
            filtered_brands.append(raw_brand.strip())
    return ', '.join(filtered_brands)

# Apply filtering row-wise
df['Filtered_Brands'] = df.apply(
    lambda row: filter_matched_brands(row['Description'], row['Matched_Brands']),
    axis=1
)

# Save output with data_key included
output_df = df[['data_key', 'Description', 'Matched_Brands', 'Filtered_Brands']]
output_df.to_excel('brand_match_results_filtered_with_key_half_desc.xlsx', index=False)

print("Filtering complete! Results saved to 'brand_match_results_filtered_with_key_half_desc.xlsx'.")


Filtering complete! Results saved to 'brand_match_results_filtered_with_key_half_desc.xlsx'.
